### Funções auxiliares

In [3]:
def formatar(x):
    """
    Formata um valor para impressão em uma matriz.

    Parâmetros:
    x: int or str - O valor a ser formatado.

    Retorna:
    str - Valor formatado, alinhado à direita e preenchido com espaços.
    """
    # Se o valor for um número inteiro, formata como inteiro alinhado à direita com um mínimo de 3 dígitos.
    if type(x) is int:
        return f"{x:>3d}"
    # Se o valor for uma string, formata como string alinhada à direita com um mínimo de 3 caracteres.
    else:
        return f"{x:>3}"

def print_matrix(S1, S2, M):
    """
    Imprime uma matriz formatada. Pode ser útil se quisermos analisar a matriz.

    Parâmetros:
    S1: list - Lista de caracteres para rótulos de coluna.
    S2: list - Lista de caracteres para rótulos de linha.
    M: list of list - Matriz a ser impressa.

    Retorna:
    None
    """
    # Obtém a largura dos rótulos de coluna
    col_width = max(len(str(x)) for x in S1)

    # Imprime os rótulos de coluna
    print(" " * (col_width + 2) + " ".join(formatar(x) for x in S1))

    # Imprime as linhas da matriz com rótulos de linha
    for x2, linha in zip(S2, M):
        # Imprime os rótulos de linha formatados à esquerda e preenchidos com espaços, seguidos pelos valores da linha formatados.
        print("{:<{}} {}".format(x2, col_width + 1, ' '.join(map(formatar, linha))))

    # Adiciona uma linha em branco após a impressão da matriz
    print()
    
def score_subst(x1, x2, g):
    """
    Calcula a pontuação de substituição entre dois caracteres.

    Parâmetros:
    x1: str - Primeiro caracter
    x2: str - Segundo caracter
    g: int - Penalidade por gap (espaço) na substituição

    Retorna:
    int - Pontuação da substituição entre os caracteres.
    """

    # Verifica se há um gap (espaço) em pelo menos um dos caracteres
    if '-' in x1 + x2:
        return g  # Retorna a penalidade por gap

    # Verifica se os caracteres fazem match, em caso afirmativo valor= 2 (exemplo)
    if x1 == x2:
        return 2  

    # Caso contrário, retorna a penalidade por mismatch valor=-1 (exemplo)
    return -1

# Algoritmo Needleman Wunsch

In [4]:
def needleman_wunsch(S1, S2, gap_penalty, score_subst):
    """
    Algoritmo de Needleman-Wunsch para calcular o alinhamento global de duas sequências.

    Parâmetros:
    S1: str - Primeira sequência
    S2: str - Segunda sequência
    gap_penalty: int - Penalidade por espaçamento (gap)
    score_subst: função auxiliar - Função que recebe dois caracteres e retorna o score de substituição entre eles

    Retorna:
    Tuple[int, str] - Pontuação do alinhamento ótimo e a sequência alinhada
    """

    
    #Garantir que recebemos duas sequências string e que o gap penalty é inteiro.
    assert isinstance(S1, str) and isinstance(S2, str), "S1 e S2 devem ser strings."
    assert isinstance(gap_penalty, int), "A penalidade por gap deve ser um número inteiro."
    assert S1.strip() != '' and S2.strip() != '', "As sequências não podem ser vazias ou consistir apenas de espaços."
    assert all(c.isalpha() for c in S1), "A sequência S1 contém caracteres inválidos."
    assert all(c.isalpha() for c in S2), "A sequência S2 contém caracteres inválidos."

    
    
    # Adiciona um espaço '-' no início de ambas as sequências
    S2 = '-' + S2
    S1 = '-' + S1

    ncols = len(S1)
    nlins = len(S2)

    # Inicializa as matrizes de scores e de trace
    scores = [[0 for _ in range(ncols)] for _ in range(nlins)]
    trace = [[0 for _ in range(ncols)] for _ in range(nlins)]

    # Inicializa a primeira linha da matriz de scores e de trace
    scores[0] = [C * gap_penalty for C, _ in enumerate(S1)]
    trace[0] = [0 if C == 0 else 'E' for C, _ in enumerate(S1)]

    # Inicializa a primeira coluna da matriz de scores
    for L, _ in enumerate(S2):
        scores[L][0] = L * gap_penalty
        trace[L][0] = 0 if L == 0 else 'C'

    # Preenche a matriz de scores e de trace tendo em conta que as primeiras linha e coluna já foram preenchidas
    for L, (X2, linha) in enumerate(zip(S2, scores)):
        for C, (X1, V) in enumerate(zip(S1, linha)):
            if L > 0 and C > 0:
                # Calcula os valores para as três opções: diagonal, esquerda, cima
                diag = scores[L - 1][C - 1] + score_subst(X1, X2, gap_penalty)
                left = scores[L][C - 1] + gap_penalty
                up = scores[L - 1][C] + gap_penalty

                # Lista de escolhas e direções correspondentes
                choices = [diag, left, up]
                # Para conseguirmos colocar acessar às letras na matriz de trace ("DiagonalEsquerdaCima")
                directions = "DEC"

                # Encontra o máximo valor e a direção correspondente
                value = max(*choices)
                # Armazena na matriz de trace a direção escolhida para a célula com base na pontuação máxima entre as opções 
                trace[L][C] = directions[choices.index(value)]
                # Armazena na matriz de pontuações a pontuação acumulada até uma célula com base na escolha da direção que maximizou a pontuação.
                scores[L][C] = value
                
    #Se quisermos visualizar as matrizes (de score e de trace)
    print_matrix(S1, S2, scores)
    print_matrix(S1, S2, trace)
    
    # Retorna a pontuação total e a sequência alinhada com recurso à função de reconstrução do alinhamento
    return scores[-1][-1], reconstruct_alignment(S1, S2, trace)


def reconstruct_alignment(S1, S2, trace):
    """
    Reconstrói a sequência alinhada a partir da matriz de trace.

    Parâmetros:
    S1: str - Primeira sequência
    S2: str - Segunda sequência
    trace: list - Matriz de rastreamento

    Retorna:
    Tuple[str, str] - Par de sequências alinhadas
    """

    aligned_seq1 = ""
    aligned_seq2 = ""
    L, C = len(S2) - 1, len(S1) - 1

    # Realiza o trace-back até atingir a primeira célula da matriz
    while trace[L][C] != 0:
        if trace[L][C] == 'D':
            aligned_seq1 = S1[C] + aligned_seq1
            aligned_seq2 = S2[L] + aligned_seq2
            L -= 1
            C -= 1
        elif trace[L][C] == 'E':
            aligned_seq1 = S1[C] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            C -= 1
        elif trace[L][C] == 'C':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = S2[L] + aligned_seq2
            L -= 1

    # Retorna o melhor par de sequências alinhadas
    return aligned_seq1, aligned_seq2

In [5]:
#Exemplo de utilização

needleman_wunsch("AGTACAT", "TTTTTTT", -1, score_subst)

     -   A   G   T   A   C   A   T
-    0  -1  -2  -3  -4  -5  -6  -7
T   -1  -1  -2   0  -1  -2  -3  -4
T   -2  -2  -2   0  -1  -2  -3  -1
T   -3  -3  -3   0  -1  -2  -3  -1
T   -4  -4  -4  -1  -1  -2  -3  -1
T   -5  -5  -5  -2  -2  -2  -3  -1
T   -6  -6  -6  -3  -3  -3  -3  -1
T   -7  -7  -7  -4  -4  -4  -4  -1

     -   A   G   T   A   C   A   T
-    0   E   E   E   E   E   E   E
T    C   D   D   D   E   E   E   D
T    C   D   D   D   D   D   D   D
T    C   D   D   D   D   D   D   D
T    C   D   D   D   D   D   D   D
T    C   D   D   D   D   D   D   D
T    C   D   D   D   D   D   D   D
T    C   D   D   D   D   D   D   D



(-1, ('AGTACAT', 'TTTTTTT'))

In [21]:
import unittest

class TestNeedlemanWunsch(unittest.TestCase):


    def test_identical_sequences(self):
        self.assertEqual(needleman_wunsch('AGTACAT', 'AGTACAT', -1, score_subst), (14, ('AGTACAT', 'AGTACAT')))

    def test_completely_different_sequences(self):
        self.assertEqual(needleman_wunsch('AAAA', 'TTTT', -1, score_subst), (-4, ('AAAA', 'TTTT')))

    def test_sequences_with_small_difference(self):
        self.assertEqual(needleman_wunsch('AGTACAT', 'AGTACGT', -1, score_subst), (11, ('AGTACAT', 'AGTACGT')))

    def test_sequences_with_large_difference(self):
        self.assertEqual(needleman_wunsch('AGTACAT', 'TTTTTTT', -1, score_subst), (-1, ('AGTACAT', 'TTTTTTT')))
        
    def test_invalid_characters(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch('AGTACAT', 'AGTAC&^%', -1, score_subst)
    
    def test_empty_sequences(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch('', '', -1, score_subst)
            
    

# Executar os testes
result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestNeedlemanWunsch))

# Verificar que falhas houveram nos testes
if not result.wasSuccessful():
    for failure in result.failures:
        print(failure)

......
----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK
